In [229]:
#imports
import pandas as pd
import os
from dotenv import load_dotenv
import boto3
from io import StringIO
from datetime import timedelta
from dateutil.relativedelta import relativedelta


# Step 1: Data Transformation


In [230]:
#load environment variables from .env file
load_dotenv()

#get aws access creds from env file
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [231]:
#create function to read in csv from AWS S3 bucket
def get_aws_csv(bucket_name:str, object_key:str)-> pd.DataFrame:
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    content = response['Body'].read().decode('utf8')
    
    return pd.read_csv(StringIO(content))

# read in patient_id_month_year.csv from S3
bucket_name='waymark-assignment'
object_key = 'patient_id_month_year.csv'
pt_df = get_aws_csv(bucket_name=bucket_name, object_key=object_key) 


/var/folders/c4/q75g9jj52ms1047cpf__wgpc0000gn/T/ipykernel_92296/3888582032.py:8: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(content))


In [232]:
#explore pt_df
print(pt_df.info())
display(pt_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047126 entries, 0 to 1047125
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  7309 non-null   object 
 1   month_year  7309 non-null   object 
 2   Unnamed: 2  0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 24.0+ MB
None


,patient_id,month_year,Unnamed: 2
0,ID0001,1/1/23,NaN
1,ID0001,2/1/23,NaN
2,ID0001,3/1/23,NaN
3,ID0001,6/1/23,NaN
4,ID0001,8/1/23,NaN
...,...,...,...
1047121,NaN,NaN,NaN
1047122,NaN,NaN,NaN
1047123,NaN,NaN,NaN
1047124,NaN,NaN,NaN


In [233]:
#grab relevant columns and drop rows with NaNs
pt_df = pt_df[['patient_id','month_year']].dropna()

#check if all day values is first of the month
print(pt_df['month_year'].apply(lambda x: x.split('/')[1]).unique())

#convert month_year to datetime
pt_df.month_year= pd.to_datetime(pt_df.month_year, format='%m/%d/%y')
pt_df

['1']


,patient_id,month_year
0,ID0001,2023-01-01
1,ID0001,2023-02-01
2,ID0001,2023-03-01
3,ID0001,2023-06-01
4,ID0001,2023-08-01
...,...,...
7304,ID1000,2023-02-01
7305,ID1000,2023-05-01
7306,ID1000,2023-07-01
7307,ID1000,2023-10-01


In [234]:
#create enrollment_end_date column by adding 1 relative month to month_year and subtracting a single day to get end of month
pt_df['enrollment_end_date'] = pt_df['month_year'].apply(lambda x: x + relativedelta(months=1) - timedelta(days=1))

#rename month_year column to enrollment_start_date since all are day = 1
pt_df.rename(columns={'month_year':'enrollment_start_date'}, inplace=True)

#drop duplicates
pt_df.drop_duplicates(inplace=True)

In [235]:
#save out pt_df to patient_enrollment_span.csv
pt_df.to_csv('patient_enrollment_span.csv',index=False)

#print number of rows
print(pt_df.shape)

display(pt_df)

(7309, 3)


,patient_id,enrollment_start_date,enrollment_end_date
0,ID0001,2023-01-01,2023-01-31
1,ID0001,2023-02-01,2023-02-28
2,ID0001,2023-03-01,2023-03-31
3,ID0001,2023-06-01,2023-06-30
4,ID0001,2023-08-01,2023-08-31
...,...,...,...
7304,ID1000,2023-02-01,2023-02-28
7305,ID1000,2023-05-01,2023-05-31
7306,ID1000,2023-07-01,2023-07-31
7307,ID1000,2023-10-01,2023-10-31


# Step 2: Data Aggregation

In [236]:
# read in outpatient_visits_file csv
bucket_name='waymark-assignment'
object_key = 'outpatient_visits_file.csv'
visit_df = get_aws_csv(bucket_name=bucket_name, object_key=object_key) 

/var/folders/c4/q75g9jj52ms1047cpf__wgpc0000gn/T/ipykernel_92296/3888582032.py:8: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(content))


In [237]:
#explore visit_df
print(visit_df.info())
display(visit_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043905 entries, 0 to 1043904
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patient_id              15328 non-null  object 
 1   date                    15328 non-null  object 
 2   outpatient_visit_count  15328 non-null  float64
 3   Unnamed: 3              0 non-null      float64
 4   Unnamed: 4              0 non-null      float64
 5   Unnamed: 5              0 non-null      float64
 6   Unnamed: 6              0 non-null      float64
 7   Unnamed: 7              0 non-null      float64
dtypes: float64(6), object(2)
memory usage: 63.7+ MB
None


,patient_id,date,outpatient_visit_count,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ID0001,1/3/23,2.0,NaN,NaN,NaN,NaN,NaN
1,ID0001,1/8/23,2.0,NaN,NaN,NaN,NaN,NaN
2,ID0001,1/9/23,2.0,NaN,NaN,NaN,NaN,NaN
3,ID0001,1/15/23,2.0,NaN,NaN,NaN,NaN,NaN
4,ID0001,1/21/23,4.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1043900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
#grab relevant columns and drop rows with NaNs
visit_df = visit_df[['patient_id', 'date', 'outpatient_visit_count']].dropna()

#convert date into datetime
visit_df['date'] = pd.to_datetime(visit_df['date'], format="%m/%d/%y")

In [239]:
#create a year_month column for joining
visit_df['year_month'] = visit_df['date'].apply(lambda x: str(x)[:7])

visit_df

,patient_id,date,outpatient_visit_count,year_month
0,ID0001,2023-01-03,2.0,2023-01
1,ID0001,2023-01-08,2.0,2023-01
2,ID0001,2023-01-09,2.0,2023-01
3,ID0001,2023-01-15,2.0,2023-01
4,ID0001,2023-01-21,4.0,2023-01
...,...,...,...,...
15323,ID0980,2023-11-25,2.0,2023-11
15324,ID0980,2023-10-20,1.0,2023-10
15325,ID0980,2023-04-04,1.0,2023-04
15326,ID0980,2023-07-26,4.0,2023-07


In [240]:
#create a year_month column on pt_df for joining
pt_df['year_month'] = pt_df['enrollment_start_date'].apply(lambda x: str(x)[:7])

#join pt_df to visit_df on patient_id and year_month
    #inner join so that only visits within an enrollment period are returned
visit_df = visit_df.merge(pt_df, how='inner', left_on=['patient_id','year_month'], right_on=['patient_id', 'year_month'])

visit_df

,patient_id,date,outpatient_visit_count,year_month,enrollment_start_date,enrollment_end_date
0,ID0001,2023-01-03,2.0,2023-01,2023-01-01,2023-01-31
1,ID0001,2023-01-08,2.0,2023-01,2023-01-01,2023-01-31
2,ID0001,2023-01-09,2.0,2023-01,2023-01-01,2023-01-31
3,ID0001,2023-01-15,2.0,2023-01,2023-01-01,2023-01-31
4,ID0001,2023-01-21,4.0,2023-01,2023-01-01,2023-01-31
...,...,...,...,...,...,...
9652,ID0980,2023-09-28,3.0,2023-09,2023-09-01,2023-09-30
9653,ID0980,2023-11-25,2.0,2023-11,2023-11-01,2023-11-30
9654,ID0980,2023-04-04,1.0,2023-04,2023-04-01,2023-04-30
9655,ID0980,2023-07-26,4.0,2023-07,2023-07-01,2023-07-31


In [241]:
#ct_outpatient_visits and ct_days_with_outpatient_visit columns
visit_df_agg = visit_df.groupby(['patient_id', 'year_month']).agg({'outpatient_visit_count':'sum',
                                                    'date': 'nunique'}).reset_index()

#rename columns
visit_df_agg.rename(columns={'outpatient_visit_count':'ct_outpatient_visits',
                             'date': 'ct_days_with_outpatient_visit'}, inplace=True)

visit_df_agg

,patient_id,year_month,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01,13.0,6
1,ID0001,2023-03,4.0,2
2,ID0001,2023-06,6.0,2
3,ID0001,2023-08,6.0,2
4,ID0001,2023-11,3.0,2
...,...,...,...,...
4527,ID0980,2023-04,1.0,1
4528,ID0980,2023-07,4.0,1
4529,ID0980,2023-09,3.0,1
4530,ID0980,2023-11,2.0,1


In [242]:
# manual QC for patient ID0001 and year_month 2023-01
visit_df[(visit_df['patient_id'].eq('ID0001'))&(visit_df['year_month'].eq('2023-01'))]

,patient_id,date,outpatient_visit_count,year_month,enrollment_start_date,enrollment_end_date
0,ID0001,2023-01-03,2.0,2023-01,2023-01-01,2023-01-31
1,ID0001,2023-01-08,2.0,2023-01,2023-01-01,2023-01-31
2,ID0001,2023-01-09,2.0,2023-01,2023-01-01,2023-01-31
3,ID0001,2023-01-15,2.0,2023-01,2023-01-01,2023-01-31
4,ID0001,2023-01-21,4.0,2023-01,2023-01-01,2023-01-31
5,ID0001,2023-01-25,1.0,2023-01,2023-01-01,2023-01-31


In [243]:
#join visit_df_agg back to pt_df
result_df = pt_df.merge(visit_df_agg, how='left', left_on=['patient_id','year_month'], right_on=['patient_id', 'year_month']).drop(columns=['year_month'])
result_df

,patient_id,enrollment_start_date,enrollment_end_date,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01-01,2023-01-31,13.0,6.0
1,ID0001,2023-02-01,2023-02-28,NaN,NaN
2,ID0001,2023-03-01,2023-03-31,4.0,2.0
3,ID0001,2023-06-01,2023-06-30,6.0,2.0
4,ID0001,2023-08-01,2023-08-31,6.0,2.0
...,...,...,...,...,...
7304,ID1000,2023-02-01,2023-02-28,NaN,NaN
7305,ID1000,2023-05-01,2023-05-31,NaN,NaN
7306,ID1000,2023-07-01,2023-07-31,NaN,NaN
7307,ID1000,2023-10-01,2023-10-31,NaN,NaN


In [244]:
#save out result_df to result.csv
result_df.to_csv('result.csv', index=False)

#print number of distinct values of ct_days_with_outpatient_visit
print(result_df.ct_days_with_outpatient_visit.nunique())

8
